<a href="https://colab.research.google.com/github/laffertybrian/Project-2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 2 - part 1 (Core)

- Brian Lafferty
- June 9th, 2022

In [1]:
# Loading datasets and libraries
import pandas as pd

space_df = pd.read_csv('/content/drive/MyDrive/07 Classification Models/nasa.csv')

machine_df = pd.read_csv('/content/drive/MyDrive/07 Classification Models/predictive_maintenance.csv')

##Dataset #1

1) Source of data: https://www.kaggle.com/datasets/shrutimehta/nasa-asteroids-classification?select=nasa.csv

In [7]:
# 2
"""A dataset of asteroids that are near to earth. Many features of the asteroids are given, 
and the goal is to predict if the astroid is hazardous to earth or not."""
space_df.head()


,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,3723955,3723955,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,2446862,2446862,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,3092506,3092506,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,3514799,3514799,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True


3) The target vector is called hazardous and is either True or False

4) This is a classification problem

5) This dataset has 39 features

6) 4687 rows

7) The data is very clean. I believe the challenges will be reviewing which features are not needed, and working with a dataset that is not balanced well could be potential problems.

##Dataset #2


1) Source of data https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification?resource=download

In [10]:
# 2
""" This dataset contains infomation about the potential machine part failure amd the features are the different 
measurements taken during use of the part"""
machine_df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


3) The target will be if the part failed during use

4) This is a classification problem

5) This dataset has 8 features

6) This dataset has 10,000 rows of data

7) This dataset is very clean and doesn't need a lot of preparation before use. It is also unblanced with very few failures. 